In [15]:
import os
import math
import time
import copy
import random
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pennylane as qml

# -------------------------
# CONFIGURATION (tweak for runtime)
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DATA_FILES = [
    "C0_dataset.csv",
    "C1_dataset.csv",
    "C2_dataset.csv",
    "C3_dataset.csv",
]
FEATURES = ["lambda", "beta", "frac_campo"]
LABEL = "ion"

# Variants to train (qubits, layers, encoder_hidden)
VARIANTS = [
    {"name": "v6_2", "n_qubits": 6, "n_layers": 2, "enc_hidden": 48},
    {"name": "v7_3", "n_qubits": 7, "n_layers": 3, "enc_hidden": 48},
    {"name": "v8_3", "n_qubits": 8, "n_layers": 3, "enc_hidden": 64},
]

KFOLD = 3
BATCH_SIZE = 128
LR_PRETRAIN = 1e-3
LR_HYBRID = 5e-4
PRETRAIN_EPOCHS = 8     # pretrain encoder classically
HYBRID_EPOCHS = 20      # hybrid training
PATIENCE = 6
DEVICE = torch.device("cpu")  # QML sim runs on CPU

# Loss tuning (gentle upweight for class-3)
MANUAL_CLASS_WEIGHTS = np.array([1.0, 1.0, 1.0, 1.2], dtype=np.float32)
FOCAL_GAMMA = 1.5

# -------------------------
# Helpers
# -------------------------
def load_and_prepare():
    df_list = [pd.read_csv(p) for p in DATA_FILES]
    df = pd.concat(df_list, ignore_index=True)
    X = df[FEATURES].values.astype(np.float32)
    y = df[LABEL].values.astype(np.int64)
    return X, y, df

def make_dataloader(X, y, batch_size=BATCH_SIZE, shuffle=True):
    X_t = torch.from_numpy(X).float()
    y_t = torch.from_numpy(y).long()
    ds = TensorDataset(X_t, y_t)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, drop_last=False)

# Focal loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        if weight is not None:
            self.register_buffer('weight', torch.tensor(weight, dtype=torch.float32))
        else:
            self.weight = None
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce = F.cross_entropy(inputs, targets, weight=self.weight, reduction='none')
        pt = torch.exp(-ce)
        loss = ((1 - pt) ** self.gamma) * ce
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# -------------------------
# Model factories (QNode + TorchLayer) per variant
# -------------------------
def create_qnode_and_torchlayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    def circuit(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    qnode = qml.qnode(dev, interface="torch")(circuit)
    weight_shapes = {"weights": (n_layers, n_qubits, 3)}
    return qnode, weight_shapes

# Encoder and Hybrid model definitions
class Encoder(nn.Module):
    def __init__(self, in_dim=3, hidden=48, latent_dim=6):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Dropout(0.12),
            nn.Linear(hidden, latent_dim)
        )
    def forward(self, x):
        return self.net(x)

class HybridQNN(nn.Module):
    def __init__(self, qnode, weight_shapes, n_qubits, n_layers, enc_hidden):
        super().__init__()
        self.encoder = Encoder(in_dim=len(FEATURES), hidden=enc_hidden, latent_dim=n_qubits)
        self.q_layer = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.head = nn.Sequential(
            nn.Linear(n_qubits, max(24, n_qubits*3)),
            nn.ReLU(),
            nn.Dropout(0.12),
            nn.Linear(max(24, n_qubits*3), 4)
        )
    def forward(self, x):
        latent = self.encoder(x)
        angles = torch.tanh(latent) * math.pi
        q_out = self.q_layer(angles)
        logits = self.head(q_out)
        return logits

# -------------------------
# Training helpers
# -------------------------
def pretrain_encoder(encoder, X_train, y_train, X_val, y_val, hidden_lr=LR_PRETRAIN, epochs=PRETRAIN_EPOCHS):
    # Train encoder + small dense head (classical) to give encoder a good initialization
    latent_dim = encoder.net[-1].out_features
    head = nn.Linear(latent_dim, 4)
    model = nn.Sequential(encoder, head).to(DEVICE)
    opt = optim.Adam(model.parameters(), lr=hidden_lr)
    loss_fn = nn.CrossEntropyLoss()
    train_loader = make_dataloader(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = make_dataloader(X_val, y_val, batch_size=BATCH_SIZE, shuffle=False)
    best_w = None
    best_acc = 0.0
    no_improve = 0
    for epoch in range(1, epochs+1):
        model.train()
        run_loss = 0.0
        for xb, yb in train_loader:
            opt.zero_grad()
            out = model(xb.to(DEVICE))
            loss = loss_fn(out, yb.to(DEVICE))
            loss.backward()
            opt.step()
            run_loss += loss.item() * xb.size(0)
        run_loss /= len(train_loader.dataset)
        # val
        model.eval()
        preds = []
        labs = []
        with torch.no_grad():
            for xb, yb in val_loader:
                out = model(xb.to(DEVICE))
                p = torch.softmax(out, dim=1)
                _, pred = torch.max(p, dim=1)
                preds.extend(pred.cpu().numpy())
                labs.extend(yb.numpy())
        acc = accuracy_score(labs, preds)
        print(f"  Pretrain epoch {epoch}/{epochs} - loss {run_loss:.4f} - val_acc {acc*100:.2f}%")
        if acc > best_acc + 1e-8:
            best_acc = acc
            best_w = copy.deepcopy(model.state_dict())
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= 5:
                break
    if best_w is not None:
        # load best weights back into the model (this updates the encoder in-place)
        model.load_state_dict(best_w)
    # encoder is the same object used inside model, so its weights are already updated; just return it
    return encoder

def train_hybrid_kfold(variant, X, y, focal_gamma=FOCAL_GAMMA, class_weights=MANUAL_CLASS_WEIGHTS):
    """
    Train a single variant with KFOLD and return OOF probs and saved model files+scalers per fold.
    """
    name = variant["name"]
    n_qubits = variant["n_qubits"]
    n_layers = variant["n_layers"]
    enc_hidden = variant["enc_hidden"]

    qnode, weight_shapes = create_qnode_and_torchlayer(n_qubits, n_layers)

    skf = StratifiedKFold(n_splits=KFOLD, shuffle=True, random_state=SEED)
    oof_probs = np.zeros((len(y), 4), dtype=np.float32)
    fold_models = []  # list of tuples (model_file, scaler)

    fold = 0
    for train_idx, val_idx in skf.split(X, y):
        fold += 1
        print(f"\nVariant {name} - Fold {fold}/{KFOLD}")
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train)
        X_val_s = scaler.transform(X_val)

        # Pretrain encoder on this fold (classical)
        encoder = Encoder(in_dim=len(FEATURES), hidden=enc_hidden, latent_dim=n_qubits)
        print(" Pretraining encoder (classical)...")
        encoder = pretrain_encoder(encoder, X_train_s, y_train, X_val_s, y_val, epochs=PRETRAIN_EPOCHS)

        # Build hybrid model and copy encoder weights
        model = HybridQNN(qnode, weight_shapes, n_qubits=n_qubits, n_layers=n_layers, enc_hidden=enc_hidden).to(DEVICE)
        # copy encoder weights from pretrain encoder (direct load of state_dict)
        model.encoder.load_state_dict(encoder.state_dict())

        loss_fn = FocalLoss(gamma=focal_gamma, weight=class_weights, reduction='mean')
        optimizer = optim.Adam(model.parameters(), lr=LR_HYBRID)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=3)

        best_val_acc = 0.0
        best_w = None
        no_improve = 0

        train_loader = make_dataloader(X_train_s, y_train, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = make_dataloader(X_val_s, y_val, batch_size=BATCH_SIZE, shuffle=False)

        for epoch in range(1, HYBRID_EPOCHS + 1):
            model.train()
            run_loss = 0.0
            for xb, yb in train_loader:
                optimizer.zero_grad()
                logits = model(xb.to(DEVICE))
                loss = loss_fn(logits, yb.to(DEVICE))
                loss.backward()
                optimizer.step()
                run_loss += loss.item() * xb.size(0)
            run_loss /= len(train_loader.dataset)

            # eval
            model.eval()
            preds = []
            probs = []
            labs = []
            with torch.no_grad():
                for xb, yb in val_loader:
                    out = model(xb.to(DEVICE))
                    p = torch.softmax(out, dim=1)
                    _, pred = torch.max(p, dim=1)
                    preds.extend(pred.cpu().numpy())
                    probs.extend(p.cpu().numpy())
                    labs.extend(yb.numpy())
            val_acc = accuracy_score(labs, preds)
            scheduler.step(val_acc)
            print(f"  Epoch {epoch}/{HYBRID_EPOCHS} - train_loss {run_loss:.4f} - val_acc {val_acc*100:.2f}%")
            if val_acc > best_val_acc + 1e-8:
                best_val_acc = val_acc
                best_w = copy.deepcopy(model.state_dict())
                no_improve = 0
                print(f"   -> new best val acc {best_val_acc*100:.2f}%")
            else:
                no_improve += 1
                if no_improve >= PATIENCE:
                    print("   -> early stopping")
                    break

        # load best, save model and scaler
        model.load_state_dict(best_w)
        model_file = f"{name}_fold{fold}.pth"
        torch.save(model.state_dict(), model_file)
        fold_models.append((model_file, scaler))

        # produce OOF probs for this fold
        model.eval()
        probs = []
        with torch.no_grad():
            for xb, yb in val_loader:
                out = model(xb.to(DEVICE))
                p = torch.softmax(out, dim=1)
                probs.append(p.cpu().numpy())
        probs = np.vstack(probs)
        oof_probs[val_idx] = probs

    return {"name": name, "variant": variant, "oof_probs": oof_probs, "fold_models": fold_models}

# -------------------------
# Ensembling / stacking helpers
# -------------------------
def ensemble_average(probs_list):
    # probs_list: list of (n_samples, n_classes)
    stacked = np.mean(np.stack(probs_list, axis=0), axis=0)
    preds = np.argmax(stacked, axis=1)
    return preds, stacked

def train_meta_and_predict(oof_prob_matrix, y):
    # oof_prob_matrix: (n_samples, num_models * n_classes)
    # train logistic regression on OOF to predict labels
    meta = LogisticRegression(max_iter=2000, multi_class='multinomial', solver='lbfgs')
    meta.fit(oof_prob_matrix, y)
    preds = meta.predict(oof_prob_matrix)
    probs = meta.predict_proba(oof_prob_matrix)
    return meta, preds, probs

# -------------------------
# MAIN PIPELINE
# -------------------------
if __name__ == "__main__":
    X, y, df = load_and_prepare()
    n_samples = len(y)
    print(f"Loaded {n_samples} samples, class counts: {np.bincount(y)}")

    variant_results = []
    # Train each variant and collect OOF probs
    for var in VARIANTS:
        print(f"\n=== Training variant {var['name']} (q={var['n_qubits']}, layers={var['n_layers']}) ===")
        res = train_hybrid_kfold(var, X, y, focal_gamma=FOCAL_GAMMA, class_weights=MANUAL_CLASS_WEIGHTS)
        variant_results.append(res)
        # quick sanity
        oof_preds = np.argmax(res["oof_probs"], axis=1)
        print(f"Variant {var['name']} OOF acc: {accuracy_score(y, oof_preds)*100:.2f}%")

    # Build OOF matrix for stacking
    probs_list = [res["oof_probs"] for res in variant_results]
    # simple average ensemble
    avg_preds, avg_probs = ensemble_average(probs_list)
    avg_acc = accuracy_score(y, avg_preds)
    print(f"\nSoft-average ensemble OOF accuracy: {avg_acc*100:.2f}%")
    print("Soft-average ensemble classification report:")
    print(classification_report(y, avg_preds, digits=4))

    # Stacking: concatenate OOF probs from each variant as features
    oof_feature_matrix = np.concatenate(probs_list, axis=1)  # shape (n_samples, num_models * n_classes)
    meta_model, meta_preds, meta_probs = train_meta_and_predict(oof_feature_matrix, y)
    meta_acc = accuracy_score(y, meta_preds)
    print(f"\nStacked (logistic) OOF accuracy: {meta_acc*100:.2f}%")
    print("Stacked classification report:")
    print(classification_report(y, meta_preds, digits=4))

    # Save artifacts
    np.save("oof_probs_variants.npy", np.stack(probs_list, axis=0))  # (num_variants, n_samples, n_classes)
    np.save("ensemble_avg_probs.npy", avg_probs)
    np.save("stacked_oof_probs.npy", meta_probs)
    # Save meta model (pickle)
    import joblib
    joblib.dump(meta_model, "meta_logistic.pkl")

    # Show confusion matrices
    print("\nConfusion matrix (soft-average):")
    print(pd.DataFrame(confusion_matrix(y, avg_preds), index=[f"True_{i}" for i in range(4)], columns=[f"Pred_{i}" for i in range(4)]))
    print("\nConfusion matrix (stacked):")
    print(pd.DataFrame(confusion_matrix(y, meta_preds), index=[f"True_{i}" for i in range(4)], columns=[f"Pred_{i}" for i in range(4)]))

    print("\nAll done. Models saved. Review OOF metrics and pick your final submission (soft-average or stacked).")


Loaded 48000 samples, class counts: [12000 12000 12000 12000]

=== Training variant v6_2 (q=6, layers=2) ===

Variant v6_2 - Fold 1/3
 Pretraining encoder (classical)...
  Pretrain epoch 1/8 - loss 1.3892 - val_acc 28.47%
  Pretrain epoch 1/8 - loss 1.3892 - val_acc 28.47%
  Pretrain epoch 2/8 - loss 1.3796 - val_acc 34.84%
  Pretrain epoch 2/8 - loss 1.3796 - val_acc 34.84%
  Pretrain epoch 3/8 - loss 1.3626 - val_acc 43.09%
  Pretrain epoch 3/8 - loss 1.3626 - val_acc 43.09%
  Pretrain epoch 4/8 - loss 1.3202 - val_acc 44.16%
  Pretrain epoch 4/8 - loss 1.3202 - val_acc 44.16%
  Pretrain epoch 5/8 - loss 1.2510 - val_acc 50.86%
  Pretrain epoch 5/8 - loss 1.2510 - val_acc 50.86%
  Pretrain epoch 6/8 - loss 1.1840 - val_acc 57.79%
  Pretrain epoch 6/8 - loss 1.1840 - val_acc 57.79%
  Pretrain epoch 7/8 - loss 1.1314 - val_acc 59.08%
  Pretrain epoch 7/8 - loss 1.1314 - val_acc 59.08%
  Pretrain epoch 8/8 - loss 1.0913 - val_acc 61.68%
  Pretrain epoch 8/8 - loss 1.0913 - val_acc 61.68

d:\QUANTUM COMPUTING\Industry Projects\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Stacked (logistic) OOF accuracy: 67.76%
Stacked classification report:
              precision    recall  f1-score   support

           0     0.5967    0.4274    0.4981     12000
           1     0.6039    0.9182    0.7286     12000
           2     0.9770    0.9823    0.9797     12000
           3     0.5048    0.3826    0.4353     12000

    accuracy                         0.6776     48000
   macro avg     0.6706    0.6776    0.6604     48000
weighted avg     0.6706    0.6776    0.6604     48000


Confusion matrix (soft-average):
        Pred_0  Pred_1  Pred_2  Pred_3
True_0    2640    2932       8    6420
True_1       0   10391       6    1603
True_2       0     722   11278       0
True_3     771    3645       2    7582

Confusion matrix (stacked):
        Pred_0  Pred_1  Pred_2  Pred_3
True_0    5129    3126      67    3678
True_1       0   11018     157     825
True_2       0     212   11788       0
True_3    3466    3890      53    4591

All done. Models saved. Review OOF metr